In [1]:
## =======================================================
## IMPORTING
## =======================================================
import os
def get_data_from_files(path):
    directory = os.listdir(path)
    results = []
    for file in directory:
        f=open(path+file,  encoding = "ISO-8859-1")
        results.append(f.read())
        f.close()
    return results


from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer = casual_tokenize)


In [31]:
# HW2
# neg = get_data_from_files('../neg_cornell/')
# pos = get_data_from_files('../pos_cornell/')

neg = get_data_from_files('../neg_hw4/')
pos = get_data_from_files('../pos_hw4/')

import pandas as pd
import numpy as np
neg_df = pd.DataFrame(neg)
pos_df = pd.DataFrame(pos)
pos_df['PoN'] = 1
neg_df['PoN'] = 0
all_df = neg_df.append(pos_df)
all_df
all_df.rename(columns={0:'text'}, inplace=True)

# # kaggle
all_df = pd.read_csv('kaggle_pos1_neg0.csv')
# all_df['0']

In [32]:
tfidf_docs = tfidf_model.fit_transform(raw_documents = all_df['text']).toarray()
print(tfidf_docs.shape)
# sms.spam.sum()

(76478, 16053)


In [33]:
mask = all_df.PoN.astype(bool).values
mask

array([False,  True,  True, ..., False, False,  True])

In [34]:
n_centroid = tfidf_docs[mask].mean(axis=0)
n_centroid

array([8.61273538e-04, 5.52010603e-05, 1.16105499e-04, ...,
       1.99336919e-05, 1.27842975e-05, 2.65665717e-05])

In [35]:
p_centroid = tfidf_docs[~mask].mean(axis=0)
p_centroid

array([2.94149033e-04, 2.05401564e-05, 2.34525098e-04, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [36]:
negativity_score = tfidf_docs.dot(n_centroid - p_centroid)
ns = negativity_score.round(2)
ns


array([ 0.01,  0.01,  0.01, ...,  0.01, -0.  , -0.  ])

In [37]:
from sklearn.preprocessing import MinMaxScaler
all_df['lda_score'] = MinMaxScaler().fit_transform(negativity_score.reshape(-1,1))
all_df['lda_predict'] = (all_df.lda_score > .5).astype(int)

In [38]:
all_df

,text,PoN,lda_score,lda_predict
0,A series of escapades demonstrating the adage ...,0,0.945184,1
1,good for the goose,1,0.922323,1
2,good,1,0.928987,1
3,"the gander , some of which occasionally amuses...",0,0.928167,1
4,amuses,1,0.899611,1
...,...,...,...,...
76473,quietly suggesting the sadness and obsession b...,0,0.913949,1
76474,sadness and obsession,0,0.914823,1
76475,sadness and,0,0.922026,1
76476,forced avuncular chortles,0,0.898606,1


In [39]:
# from pugnlp.stats import Confusion
# Confusion(all_df['PoN lda_predict'.split()])

In [40]:
(1. - (all_df.PoN - all_df.lda_predict).abs().sum() / len(all_df)).round(3)


0.643

In [41]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(all_df['PoN'], all_df['lda_predict'])

In [42]:
cm

array([[ 7072, 27273],
       [    0, 42133]])

In [43]:
cd = pd.read_csv('cats_and_dogs_sorted.csv')

VOCABULARY = vocabulary='cat dog apple lion NYC love'.lower().split()
DOCS = cd['0'].values
def docs_to_tdm(docs=DOCS, vocabulary=VOCABULARY, verbosity=0):
    tfidfer = TfidfVectorizer(min_df=1, max_df=.99, stop_words=None, token_pattern=r'(?u)\b\w+\b',
                              vocabulary=vocabulary)
    tfidf_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    id_words = [(i, w) for (w, i) in tfidfer.vocabulary_.items()]
    tfidf_dense.columns = list(zip(*sorted(id_words)))[1]

    tfidfer.use_idf = False
    tfidfer.norm = None
    bow_dense = pd.DataFrame(tfidfer.fit_transform(docs).todense())
    bow_dense.columns = list(zip(*sorted(id_words)))[1]
    bow_dense = bow_dense.astype(int)
    tfidfer.use_idf = True
    tfidfer.norm = 'l2'
    if verbosity:
        print(tfidf_dense.T)
    return bow_dense.T, tfidf_dense.T, tfidfer



In [44]:
a, b, c = docs_to_tdm()
a


,0,1,2,3,4,5,6,7,8,9,...,253,254,255,256,257,258,259,260,261,262
cat,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
dog,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
apple,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
lion,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
nyc,1,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
love,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
def lsa_models(vocabulary='cat dog apple lion NYC love'.lower().split(), docs=11, verbosity=0):
    # vocabulary = 'cat dog apple lion NYC love big small bright'.lower().split()
    if isinstance(docs, int):
        docs = DOCS
    tdm, tfidfdm, tfidfer = docs_to_tdm(docs=docs, vocabulary=vocabulary)
    lsa_bow_model = lsa(tdm)  # (tdm - tdm.mean(axis=1)) # SVD fails to converge if you center, like PCA does
    lsa_bow_model['vocabulary'] = tdm.index.values
    lsa_bow_model['docs'] = docs
    err = accuracy_study(verbosity=verbosity, **lsa_bow_model)
    lsa_bow_model['err'] = err
    lsa_bow_model['accuracy'] = list(1. - np.array(err))
    
    lsa_tfidf_model = lsa(tdm=tfidfdm)
    lsa_bow_model['vocabulary'] = tfidfdm.index.values
    lsa_tfidf_model['docs'] = docs
    err = accuracy_study(verbosity=verbosity, **lsa_tfidf_model)
    lsa_tfidf_model['err'] = err
    lsa_tfidf_model['accuracy'] = list(1. - np.array(err))

    return lsa_bow_model, lsa_tfidf_model

In [46]:
def lsa(tdm, verbosity=0):
    if verbosity:
        print(tdm)
        #         0   1   2   3   4   5   6   7   8   9   10
        # cat     0   0   0   0   0   0   1   1   1   0   1
        # dog     0   0   0   0   0   0   0   0   0   0   1
        # apple   1   1   0   1   1   1   0   0   0   0   0
        # lion    0   0   0   0   0   0   0   1   0   0   0
        # love    0   0   1   0   0   0   0   0   1   1   0
        # nyc     1   1   1   1   1   0   0   0   0   1   0

    u, s, vt = np.linalg.svd(tdm)

    u = pd.DataFrame(u, index=tdm.index)
    if verbosity:
        print('U')
        print(u.round(2))
        # U
        #           0     1     2     3     4     5
        # cat   -0.04  0.83 -0.38 -0.00  0.11  0.38
        # dog   -0.00  0.21 -0.18 -0.71 -0.39 -0.52
        # apple -0.62 -0.21 -0.51  0.00  0.49 -0.27
        # lion  -0.00  0.21 -0.18  0.71 -0.39 -0.52
        # love  -0.22  0.42  0.69  0.00  0.41 -0.37
        # nyc   -0.75  0.00  0.24 -0.00 -0.52  0.32

    vt = pd.DataFrame(vt, index=['d{}'.format(i) for i in range(len(vt))])
    if verbosity:
        print('VT')
        print(vt.round(2))
        # VT
        #        0     1     2     3     4     5     6     7     8     9     10
        # d0  -0.44 -0.44 -0.31 -0.44 -0.44 -0.20 -0.01 -0.01 -0.08 -0.31 -0.01
        # d1  -0.09 -0.09  0.19 -0.09 -0.09 -0.09  0.37  0.47  0.56  0.19  0.47
        # d2  -0.16 -0.16  0.52 -0.16 -0.16 -0.29 -0.22 -0.32  0.17  0.52 -0.32
        # d3   0.00 -0.00  0.00  0.00  0.00  0.00 -0.00  0.71  0.00  0.00 -0.71
        # d4  -0.04 -0.04 -0.14 -0.04 -0.04  0.58  0.13 -0.33  0.62 -0.14 -0.33
        # d5   0.09  0.09 -0.10  0.09  0.09 -0.51  0.73 -0.27  0.01 -0.10 -0.27
        # d6  -0.55  0.24  0.15  0.36 -0.38  0.32  0.32  0.00 -0.32  0.17  0.00
        # d7  -0.32  0.46  0.23 -0.64  0.41  0.09  0.09  0.00 -0.09 -0.14  0.00
        # d8  -0.52  0.27 -0.24  0.39  0.22 -0.36 -0.36 -0.00  0.36 -0.12  0.00
        # d9  -0.14 -0.14 -0.58 -0.14  0.32  0.10  0.10 -0.00 -0.10  0.68 -0.00
        # d10 -0.27 -0.63  0.31  0.23  0.55  0.12  0.12 -0.00 -0.12 -0.19 -0.00

    # Reconstruct the original term-document matrix.
    # The sum of the squares of the error is 0.

    return {'u': u, 's': s, 'vt': vt, 'tdm': tdm}

In [47]:
def accuracy_study(tdm=None, u=None, s=None, vt=None, verbosity=0, **kwargs):
    """ Reconstruct the term-document matrix and measure error as SVD terms are truncated
    """
    smat = np.zeros((len(u), len(vt)))
    np.fill_diagonal(smat, s)
    smat = pd.DataFrame(smat, columns=vt.index, index=u.index)
    if verbosity:
        print()
        print('Sigma:')
        print(smat.round(2))
        print()
        print('Sigma without zeroing any dim:')
        print(np.diag(smat.round(2)))
    tdm_prime = u.values.dot(smat.values).dot(vt.values)
    if verbosity:
        print()
        print('Reconstructed Term-Document Matrix')
        print(tdm_prime.round(2))

    err = [np.sqrt(((tdm_prime - tdm).values.flatten() ** 2).sum() / np.product(tdm.shape))]
    if verbosity:
        print()
        print('Error without reducing dimensions:')
        print(err[-1])
    # 2.3481474529927113e-15

    smat2 = smat.copy()
    for numdim in range(len(s) - 1, 0, -1):
        smat2.iloc[numdim, numdim] = 0
        if verbosity:
            print('Sigma after zeroing out dim {}'.format(numdim))
            print(np.diag(smat2.round(2)))
            #           d0    d1   d2   d3   d4   d5
            # ship    2.16  0.00  0.0  0.0  0.0  0.0
            # boat    0.00  1.59  0.0  0.0  0.0  0.0
            # ocean   0.00  0.00  0.0  0.0  0.0  0.0
            # voyage  0.00  0.00  0.0  0.0  0.0  0.0
            # trip    0.00  0.00  0.0  0.0  0.0  0.0

        tdm_prime2 = u.values.dot(smat2.values).dot(vt.values)
        err += [np.sqrt(((tdm_prime2 - tdm).values.flatten() ** 2).sum() / np.product(tdm.shape))]
        if verbosity:
            print('Error after zeroing out dim {}'.format(numdim))
            print(err[-1])
    return err

In [48]:
lsa_bow, lsa_tdif_bow = lsa_models()

In [49]:
lsa_bow

{'u':               0         1         2         3         4         5
 cat   -0.963953  0.256998  0.039468  0.022509  0.051557 -0.004949
 dog   -0.255938 -0.965615  0.043947 -0.007358 -0.003217 -0.009401
 apple -0.004551 -0.004413 -0.294570  0.179873 -0.006208 -0.938507
 lion  -0.048551  0.016315  0.006249  0.012370 -0.998566  0.007174
 nyc   -0.045182 -0.034756 -0.930156  0.163649  0.000160  0.323695
 love  -0.029525  0.006817 -0.210973 -0.969614 -0.012643 -0.119421,
 's': array([6.6864976 , 6.04182871, 4.19889332, 2.93925665, 2.21285739,
        1.90273045]),
 'vt':            0         1         2         3         4         5         6    \
 d0   -0.007438 -0.007438 -0.011173 -0.007438 -0.007438 -0.000681 -0.144164   
 d1   -0.006483 -0.006483 -0.004624 -0.006483 -0.006483 -0.000730  0.042537   
 d2   -0.291678 -0.291678 -0.271769 -0.291678 -0.291678 -0.070154  0.009400   
 d3    0.116874  0.116874 -0.274207  0.116874  0.116874  0.061197  0.007658   
 d4   -0.002733 -0.002733 -0.